In [1]:
YEAR = None

In [2]:
# Parameters
YEAR = 2008


In [3]:
arg_year = YEAR if isinstance(YEAR, int) else 1999

In [4]:
# !pip install requests requests-html pandas

In [5]:
from dataclasses import dataclass
import pathlib
import pandas as pd
import requests
from requests_html import HTML

In [6]:
@dataclass
class ScrapeBoxOffice:
    base_endpoint:str = "https://www.boxofficemojo.com/year/world/"
    year:int = None
    save_raw:bool = False
    save:bool = False
    output_dir: str = "."
    table_selector: str = '.imdb-scroll-table'
    table_data = []
    table_header_names = []
    df = pd.DataFrame()
    
    @property
    def name(self):
        return self.year if isinstance(self.year, int) else 'world'
    
    def get_endpoint(self):
        endpoint = self.base_endpoint
        if isinstance(self.year, int):
            endpoint = f"{endpoint}{self.year}/"
        return endpoint
    
    def get_output_dir(self):
        return pathlib.Path(self.output_dir)
    
    def extract_html_str(self, endpoint=None):
        url = endpoint if endpoint is not None else self.get_endpoint()
        r = requests.get(url, stream=True)
        html_text = None
        status = r.status_code
        if r.status_code == 200:
            html_text = r.text
            if self.save_raw:
                output_fname = f"{self.name}.html"
                raw_output_dir = self.get_output_dir() / 'html'
                raw_output_dir.mkdir(exist_ok=True, parents=True)
                output_fname = raw_output_dir / output_fname
                with open(f"{output_fname}", 'w') as f:
                    f.write(html_text)
            return html_text, status
        return html_text, status
    
    def parse_html(self, html_str=''):
        r_html = HTML(html=html_str)
        r_table = r_html.find(self.table_selector)
        if len(r_table) == 0:
            return None
        table_data = []
        header_names = []
        parsed_table = r_table[0]
        rows = parsed_table.find("tr")
        header_row = rows[0]
        header_cols = header_row.find('th')
        header_names = [x.text for x in header_cols]
        for row in rows[1:]:
            cols = row.find("td")
            row_data = []
            row_dict_data = {}
            for i, col in enumerate(cols):
                header_name = header_names[i]
                row_data.append(col.text)
            table_data.append(row_data)
        self.table_data = table_data
        self.table_header_names = header_names
        return self.table_data, self.table_header_names
    
    def to_df(self, data=[], columns=[]):
        return pd.DataFrame(data, columns=columns)
    
    def run(self, save=False):
        save = self.save if save is False else save
        endpoint = self.get_endpoint()
        html_str, status = self.extract_html_str(endpoint=endpoint)
        if status not in range(200, 299):
            raise Exception(f"Extraction failed, endpoint status {status} at {endpoint}")
        data, headers = self.parse_html(html_str if html_str is not None else '')
        df = self.to_df(data=data, columns=headers)
        self.df = df
        if save:
            filepath = self.get_output_dir() / f'{self.name}.csv'
            df.to_csv(filepath, index=False)
        return self.df

In [7]:
scraper = ScrapeBoxOffice(year=arg_year, save=True, save_raw=True, output_dir='data')
df = scraper.run()
df.head()

,Rank,Release Group,Worldwide,Domestic,%,Foreign,%
0,1,The Dark Knight,"$1,003,045,358","$533,345,358",53.2%,"$469,700,000",46.8%
1,2,Indiana Jones and the Kingdom of the Crystal S...,"$790,653,942","$317,101,119",40.1%,"$473,552,823",59.9%
2,3,Kung Fu Panda,"$631,744,560","$215,434,591",34.1%,"$416,309,969",65.9%
3,4,Hancock,"$629,443,428","$227,946,274",36.2%,"$401,497,154",63.8%
4,5,Mamma Mia!,"$609,841,637","$144,130,063",23.6%,"$465,711,574",76.4%
